In [28]:
import nest_asyncio
from autorag.data.qa.filter.passage_dependency import passage_dependency_filter_llama_index

nest_asyncio.apply()

In [29]:
from dotenv import load_dotenv

load_dotenv()

True

In [30]:
import pandas as pd
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

## Custom Prompt를 사용하는 방법을 알아봅시다.

Human in the loop의 핵심 => Custom Prompt를 활용해 보다 좋은 질문 생성법을 만드는 방식

In [31]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from autorag.data.qa.query.llama_gen_query import llama_index_generate_base
from llama_index.core.base.llms.base import BaseLLM
from typing import Dict


async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content="""
주어진 정보를 바탕으로 질문을 생성하세요.			
""",
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [32]:
corpus_data_path = os.path.join(data_dir, 'data_creation', 'corpus.parquet')
corpus_df = pd.read_parquet(corpus_data_path, engine='pyarrow')

In [56]:
from autorag.data.qa.sample import random_single_hop
from autorag.data.qa.schema import Corpus

initial_corpus = Corpus(corpus_df)
initial_qa = initial_corpus.sample(random_single_hop, n=15).map(
        lambda df: df.reset_index(drop=True),
    ).make_retrieval_gt_contents()


In [57]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents
0,8c0c9613-3952-4f8b-8df4-ab3b8ba0aa2e,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점..."
1,a97aa280-aef7-405d-b09a-1fd3564a8a32,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,..."
2,a30f140c-c32e-4c52-8afa-69c479706ef8,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...
3,8759a2ef-8537-4a47-aa1a-b76132cb4f43,[[ef778bb8-8bdc-4245-bbfd-c48c43a4c00a]],[[4 신청 및 접수\n□ 신청 ‧ 접수 기간\n◦ 접수기간 : 2024. 5. 7...
4,9c4fb76e-78c2-4666-a2e6-7dd5afd32e0f,[[cb840303-3053-47e0-b142-c622d79187f5]],[[14 -]]
5,5c72433c-9678-4fc4-8112-0e0aeb48ffb3,[[f9cfcc4f-8e7c-44e9-a931-5bfe5d5dd90d]],[[□ 의무 및 역할\n◦ 선정자는「중소기업창업 지원사업 운영요령」 및 「창업사업화...
6,0ce202b8-e596-4f73-8876-339aa0642199,[[228d107f-edcc-49df-93ce-2e9d64d51c29]],[[# 6 유의사항◈창업지원사업에신청하는창업기업은관련법령에따라 다음사항에유의하여야합...
7,a632d3d6-f07a-4b87-a7a6-d64fcff0615b,[[3b32c764-c9b6-46b3-b3b3-01c43ad7ee3b]],[[# 붙임]]
8,4095b13b-8c75-4047-81a1-86b31cdb32eb,[[4acb4bb2-b6b5-4eb8-a3a0-b1f15d7a9d78]],[[# 창업여부 기준표|신청기업구분|상세구분| |창업여부|\n|---|---|---...
9,10254b84-2eff-4f72-801f-f7117e1c8b59,[[e0341a6a-4135-4cf1-8bd0-dfceeb052bc2]],[[※세부일정은대내‧외사정에의해변경될수있음|공고|창업기업신청‧접수|요건검토및선정평가...


Custom Prompt를 적용합니다

In [54]:
from llama_index.llms.openai import OpenAI

llm = OpenAI()
first_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/16/25 17:26:27] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=212225;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=694289;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=250082;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=866827;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=15115;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=538592;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [36]:
first_qa.data['query']

0    해외VC로 투자를 유치하기 위한 조건 중에는 어떤 요건들이 있는지 설명해 주실 수 ...
1    어떤 분야의 기업들이 해외 VC 투자를 받을 수 있는 조건이나 지원을 받을 수 있는...
2    창업기업 선정평가의 절차는 무엇이며, 최종 선정자격을 얻기 위해 어떤 단계를 거쳐야...
Name: query, dtype: object

retrieval gt에 알맞은 질문이 생성되었는지 확인해 봅니다.

In [37]:
def visualize_query_retrieval_gt(df, idx: int):
	print(f'질문 : {df.data.iloc[idx]["query"]}')
	print(f"단락 내용:\n{df.data.iloc[idx]['retrieval_gt_contents'][0][0]}")

In [38]:
visualize_query_retrieval_gt(first_qa, 0)

질문 : 해외VC로 투자를 유치하기 위한 조건 중에는 어떤 요건들이 있는지 설명해 주실 수 있나요?
단락 내용:
본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점을 기준으로
요건검토 예정
▶ 해외VC (투자사)
① 해외VC 요건(필수)
• 해외VC의 본사 소재지가 ‘한국’이 아닌 ‘국외’
- 해외VC의 소재지는 요건검토 시 적정 여부 판단
• 모집공고일 기준으로 최근 3년 이내(2021.5.8. ~ 2024.5.7.)의 해외펀드 운용 실적 보유
• 전문인력(벤처캐피탈 및 액셀러레이터 등 투자 관련 경력 2년 이상) 1명 이상 보유
• (우대)‘한국’ 지사·사무소 보유 또는 글로벌 펀드 출자 VC
② 투자유치 조건(투자방식은 요건 검토 시 적정 여부 판단)
• 최근 3년 이내 글로벌 팁스 사업신청 창업기업에게 20만불 이상 투자
- 모집공고일 기준으로 투자금 납입일이 3년 이내(2021.5.8. ~ 2024.5.7.)
- 해외VC 투자금액의 총 합산액(국내VC의 투자는 미인정)(외화기준)
- 신규 투자 또는 신규 및 기존지분 인수 등 혼합투자(단, 신규50% 이상) 모두 인정(금액기준)
- Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 인정 불가
- USD 외 투자 시, 모집공고일 기준 서울외국환중개 매매기준율 미국달러(USD) 적용
- 2 -


In [23]:
visualize_query_retrieval_gt(first_qa, 1)

질문 : 어떤 분야의 기업들이 해외 VC 투자를 받을 수 있는 조건이나 지원을 받을 수 있는 프로그램에 대해 언급되고 있나요?
단락 내용:
1. ①팁스성공기업 ,시드포함)(프리
2. ②글로벌지원사업(글로벌액셀러레이팅,글로벌창업사관학교, 해외실증(PoC),KSC등)성공(졸업)기업
3. ③전략기술분야 ‧부품‧장비,10대초격차분야)붙임5‘해외VC글로벌’펀드(한국벤처투자출자)투자자
5. ⑤(해외VC)국내지사·사무소보유여부
6. ⑥(해외VC)투자전략및지원계획발표# 주요 참고사항*사업계획서양식:“[별첨1]사업계획서서식”참조**해외VC추천서:“[별첨2]증빙서류제출목록”참조***반드시동사업의사업계획서양식을활용 외양식제출시평가대상에서제외****가점관련증빙서류는사업신청시제출하며,미제출시가점불인정*****창업기업여부확인을위하여추가적인서류를요청할수있으며,기존에발급받은‘창업기업확인서’보유시해당서류로확인대체가능(단,사업공고일기준확인서상의유효기간이경과되지않았어야함)


### 도메인에 대한 정보를 추가해봅시다.

In [39]:
async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content="""당신은 스타트업의 지원 프로그램에 대해 검색할 수 있는 RAG 시스템의 관리자입니다.
RAG 시스템을 관리를 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 다양한 스타트업 지원 프로그램에 대한 문서입니다.
문서를 보고 예상되는 사용자들의 질문을 적으세요.
""",
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [40]:
second_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/16/25 17:18:47] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=947514;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=99825;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/16/25 17:18:48] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=528370;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=663203;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=424560;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=571662;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [41]:
second_qa.data['query']

0    1. 해외VC 지원을 받기 위한 요건은 무엇인가요?\n2. 해외VC가 투자한 금액의...
1    1. 어떤 종류의 스타트업 지원 프로그램이 있나요?\n2. 글로벌 창업을 지원하는 ...
2    1. 창업기업 선정 평가는 몇 단계로 이루어지나요?\n2. 창업기업 선정 평가의 각...
Name: query, dtype: object

In [42]:
visualize_query_retrieval_gt(second_qa, 0)

질문 : 1. 해외VC 지원을 받기 위한 요건은 무엇인가요?
2. 해외VC가 투자한 금액의 조건은 어떻게 되나요?
3. 해외VC의 본사 소재지가 한국이 아니어도 지원을 받을 수 있나요?
4. 해외VC 투자금액의 총 합산액은 어떻게 계산되나요?
5. Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 어떤 조건이 적용되나요?
단락 내용:
본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점을 기준으로
요건검토 예정
▶ 해외VC (투자사)
① 해외VC 요건(필수)
• 해외VC의 본사 소재지가 ‘한국’이 아닌 ‘국외’
- 해외VC의 소재지는 요건검토 시 적정 여부 판단
• 모집공고일 기준으로 최근 3년 이내(2021.5.8. ~ 2024.5.7.)의 해외펀드 운용 실적 보유
• 전문인력(벤처캐피탈 및 액셀러레이터 등 투자 관련 경력 2년 이상) 1명 이상 보유
• (우대)‘한국’ 지사·사무소 보유 또는 글로벌 펀드 출자 VC
② 투자유치 조건(투자방식은 요건 검토 시 적정 여부 판단)
• 최근 3년 이내 글로벌 팁스 사업신청 창업기업에게 20만불 이상 투자
- 모집공고일 기준으로 투자금 납입일이 3년 이내(2021.5.8. ~ 2024.5.7.)
- 해외VC 투자금액의 총 합산액(국내VC의 투자는 미인정)(외화기준)
- 신규 투자 또는 신규 및 기존지분 인수 등 혼합투자(단, 신규50% 이상) 모두 인정(금액기준)
- Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 인정 불가
- USD 외 투자 시, 모집공고일 기준 서울외국환중개 매매기준율 미국달러(USD) 적용
- 2 -


하나씩만 생성하도록 합니다

In [43]:
async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content="""당신은 스타트업의 지원 프로그램에 대해 검색할 수 있는 RAG 시스템의 관리자입니다.
RAG 시스템을 관리를 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 다양한 스타트업 지원 프로그램에 대한 문서입니다.
문서를 보고 예상되는 사용자들의 질문을 적으세요.
단, 반드시 하나의 질문만을 생성하세요.
생성하는 질문에는 미사여구 없이, 해당 질문만이 포함되어야 합니다.
""",
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [44]:
third_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/16/25 17:20:36] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=458727;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=233399;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=195982;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=726748;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=401482;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=312273;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [45]:
third_qa.data['query']

0         해외VC 지원 프로그램에 대한 자격 요건은 무엇인가요?
1    해외 VC 투자를 받기 위한 사업계획서 제출 방법은 무엇인가요?
2                   평가 결과 발표는 언제 이루어지나요?
Name: query, dtype: object

In [47]:
visualize_query_retrieval_gt(third_qa, 2)

질문 : 평가 결과 발표는 언제 이루어지나요?
단락 내용:
# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표*선정평가운영및평가일정,평가결과등은주관기관에서안내예정# 창업기업선정평가절차(안)1. 요건검토
2. 서류평가
3. 발표평가
4. 최종선정자격기준검토및 서류평가대상자 확정제출한사업계획서 토대로서면평가창업기업대표자 발표및질의응답 심의후최종공고~’24.6.7예정 ’24.6.12~6.13 ’24.6.18.~6.20 ’24.7월.예정신청기업요건검토는창업진흥원및주관기관에서상시진행하며, 선정‧협약이후에도신청자격미충족등이확인되는경우탈락처리예정※상기일정은신청기업의수등대내외사정에따라변경될수있음#


## Passage Dependent

Filter 적용 혹은 질문 생성 바꾸기

In [58]:
async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content="""당신은 스타트업의 지원 프로그램 중 하나인 "글로벌 팁스 프로그램"에 대해 알려주는 어시스턴트의 관리자입니다.
RAG 시스템을 관리를 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 글로벌 팁스 프로그램에 대한 안내 문서입니다.
문서를 보고 예상되는 사용자들의 질문을 적으세요.
단, 반드시 하나의 질문만을 생성하세요.
생성하는 질문에는 미사여구 없이, 해당 질문만이 포함되어야 합니다.
""",
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [59]:
fourth_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/16/25 17:27:29] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=36066;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=982740;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=365260;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=416661;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=962827;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=366354;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=232776;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=934194;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=973062;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=123761;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=318016;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=151999;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=690426;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=185503;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/16/25 17:27:30] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=967096;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=262569;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=900140;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=589813;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=492003;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=708395;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [60]:
fourth_qa.data['query']

0           해외VC로서 글로벌 팁스 프로그램에 지원하기 위한 최소한의 요건은 무엇인가요?
1                        해외 VC 투자를 받기 위해 필요한 서류는 무엇인가요?
2                                평가 결과는 어떻게 안내받을 수 있나요?
3                  접수 마감일 이후에는 사업계획서 등 일체 내용 수정이 가능한가요?
4             What does "14 -" refer to in the program?
5     선정자는 어떤 경우에 현금을 창업진흥원이 정한 기한 내에 지정한 계좌에 입금해야 하나요?
6                   창업지원사업에 신청하는 기업은 어떤 유의사항을 준수해아해합니까?
7                           글로벌 팁스 프로그램에는 어떤 혜택이 제공되나요?
8                             창업자로서의 자격을 어떻게 확인할 수 있나요?
9                         창업기업은 언제부터 언제까지 사업을 수행해야 하나요?
10                   어떤 경우에 글로벌 팁스 프로그램의 선정이 취소될 수 있나요?
11                        어떤 서류를 제출해야 하며, 제출 방법은 무엇인가요?
12                       사업계획서 작성 시 주의해야 할 유의사항은 무엇인가요?
13                  어떤 품목들이 "글로벌 팁스 프로그램"에서 지원되는 대상인가요?
14           어떤 분야의 소재, 부품, 장비가 글로벌 팁스 프로그램에서 지원되고 있나요?
Name: query, dtype: object

In [61]:
visualize_query_retrieval_gt(fourth_qa, 0)

질문 : 해외VC로서 글로벌 팁스 프로그램에 지원하기 위한 최소한의 요건은 무엇인가요?
단락 내용:
본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점을 기준으로
요건검토 예정
▶ 해외VC (투자사)
① 해외VC 요건(필수)
• 해외VC의 본사 소재지가 ‘한국’이 아닌 ‘국외’
- 해외VC의 소재지는 요건검토 시 적정 여부 판단
• 모집공고일 기준으로 최근 3년 이내(2021.5.8. ~ 2024.5.7.)의 해외펀드 운용 실적 보유
• 전문인력(벤처캐피탈 및 액셀러레이터 등 투자 관련 경력 2년 이상) 1명 이상 보유
• (우대)‘한국’ 지사·사무소 보유 또는 글로벌 펀드 출자 VC
② 투자유치 조건(투자방식은 요건 검토 시 적정 여부 판단)
• 최근 3년 이내 글로벌 팁스 사업신청 창업기업에게 20만불 이상 투자
- 모집공고일 기준으로 투자금 납입일이 3년 이내(2021.5.8. ~ 2024.5.7.)
- 해외VC 투자금액의 총 합산액(국내VC의 투자는 미인정)(외화기준)
- 신규 투자 또는 신규 및 기존지분 인수 등 혼합투자(단, 신규50% 이상) 모두 인정(금액기준)
- Co-GP의 경우, GP의 구성이 국내 VC로만 이루어진 경우 인정 불가
- USD 외 투자 시, 모집공고일 기준 서울외국환중개 매매기준율 미국달러(USD) 적용
- 2 -


In [64]:
from autorag.data.qa.filter.passage_dependency import passage_dependency_filter_llama_index

filtered_qa = fourth_qa.batch_filter(
	passage_dependency_filter_llama_index,
	llm=llm,
	lang="ko",
)

[01/16/25 17:27:49] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=277678;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=239838;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=291140;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=958156;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=515155;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=505743;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=706493;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=225642;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=9303;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=240320;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=228660;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=284178;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=235978;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=507890;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=516687;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=477798;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=361146;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=234969;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=847696;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=928417;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [65]:
filtered_qa.data['query']

Series([], Name: query, dtype: object)

In [ ]:
# Persona Evolving
# Do it on your own